In [385]:
import numpy as np
import pandas as pd 
import pickle
from sklearn import set_config
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import OrdinalEncoder
# from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler 
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score

In [386]:
df = pd.read_csv('MBA.csv')

In [387]:
df.head()

,application_id,gender,international,gpa,major,race,gmat,work_exp,work_industry,admission
0,1,Female,False,3.30,Business,Asian,620.0,3.0,Financial Services,Admit
1,2,Male,False,3.28,Humanities,Black,680.0,5.0,Investment Management,NaN
2,3,Female,True,3.30,Business,NaN,710.0,5.0,Technology,Admit
3,4,Male,False,3.47,STEM,Black,690.0,6.0,Technology,NaN
4,5,Male,False,3.35,STEM,Hispanic,590.0,5.0,Consulting,NaN


In [388]:
df.isnull().sum()

application_id       0
gender               0
international        0
gpa                  0
major                0
race              1842
gmat                 0
work_exp             0
work_industry        0
admission         5194
dtype: int64

In [389]:
df['major'].value_counts()

major
Humanities    2481
STEM          1875
Business      1838
Name: count, dtype: int64

In [390]:
df.drop(columns=['application_id','international', 'race', 'work_industry', 'gpa', 'admission'], inplace=True)
df

,gender,major,gmat,work_exp
0,Female,Business,620.0,3.0
1,Male,Humanities,680.0,5.0
2,Female,Business,710.0,5.0
3,Male,STEM,690.0,6.0
4,Male,STEM,590.0,5.0
...,...,...,...,...
6189,Male,Business,640.0,5.0
6190,Male,STEM,670.0,4.0
6191,Female,Business,680.0,5.0
6192,Male,Business,590.0,5.0


In [391]:
# df.replace('Admit',1,inplace=True)
# df.fillna(0, inplace=True)
# df

In [392]:
df.dtypes

gender       object
major        object
gmat        float64
work_exp    float64
dtype: object

In [393]:
df['gmat']=df['gmat'].astype('int')
df['work_exp']=df['work_exp'].astype('int')
df.dtypes

gender      object
major       object
gmat         int32
work_exp     int32
dtype: object

In [394]:
# train_test_split 
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['work_exp']), 
                                                    df['work_exp'], 
                                                    test_size = 0.2,
                                                    random_state=42)
x_train.head()

,gender,major,gmat
6189,Male,Business,640
711,Female,Humanities,690
3954,Female,STEM,660
1020,Female,Humanities,670
5726,Male,Business,640


In [395]:
#Applying imputation 
# trf1 = ColumnTransformer([
#     ('StudyHoursPerWeek', SimpleImputer(strategy='most_frequent'), [1])],
#     remainder='passthrough')

In [396]:
# One Hot Encoding
trf1 = ColumnTransformer([
    ('gender', OneHotEncoder(sparse_output = False, handle_unknown='ignore'), [0])],
remainder='passthrough')

In [397]:
# Ordinal Encoding
trf2 = ColumnTransformer([
    ('major', OrdinalEncoder(categories=[['STEM','Humanities', 'Business']]), [1])],
remainder='passthrough')

In [398]:
# # Label Encoding
# trf3 = ColumnTransformer([
#     ('admission', LabelEncoder(), [4])],
# remainder='passthrough')

In [399]:
# Scaling
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,3))
])

In [400]:
# Feature Selection 
trf4 = SelectKBest(score_func=chi2, k=2)

In [401]:
trf5 = DecisionTreeClassifier()

In [402]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

In [403]:
# pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5,trf6)

In [404]:
# training
pipe.fit(x_train,y_train)

ValueError: Found unknown categories [0.0, 1.0] in column 0 during fit

In [407]:
set_config(display='diagram')

In [408]:
pipe.named_steps['trf1'].transformers_[0][1]

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [409]:
y_pred = pipe.predict(x_test)

AttributeError: 'ColumnTransformer' object has no attribute 'transformers_'

In [410]:
accuracy_score(y_test,y_pred)

NameError: name 'y_pred' is not defined

In [406]:
cross_val_score(pipe, x_train, y_train, cv=5, scoring='accuracy').mean()

C:\Users\dell\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\dell\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\dell\anaconda3\lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\dell\anaconda3\lib\site-packages\sklearn\pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "C:\Users\dell\anaconda3\lib\site-packages\joblib\memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "C:\Users\dell\anaconda3\lib\site-packages\sklearn\pipeline.py", l

In [ ]:
# gridsearch

params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [ ]:
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')

In [ ]:
grid.fit(x_train,y_train)

In [411]:
grid.best_score_

NameError: name 'grid' is not defined

In [412]:
grid.best_params_

NameError: name 'grid' is not defined

In [ ]:
pickle.dump(pipe,open('pipe.pkl','wb'))